In [30]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader

from torchvision import datasets, transforms
# from torchvision.transforms import ToTensor, Lambda, Compose
import matplotlib.pyplot as plt


labels_dict = {
    0: 'T-shirt/top',
    1: 'Trouser',
    2: 'Pullover',
    3: 'Dress',
    4: 'Coat',
    5: 'Sandal',
    6: 'Shirt',
    7: 'Sneaker',
    8: 'Bag',
    9: 'Ankle boot'
}


In [31]:
training_data = datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=transforms.ToTensor()
)

test_data = datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=transforms.ToTensor()
)


In [32]:
batch_size = 128
train_dataloader = DataLoader(training_data, batch_size=batch_size)
test_dataloader = DataLoader(test_data, batch_size=batch_size)


device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using {device} device")

Using cuda device


In [33]:

print("The train set contains {} images, in {} batches".format(len(train_dataloader.dataset), len(train_dataloader)))
# print("The validation set contains {} images, in {} batches".format(len(valid_loader.dataset), len(valid_loader)))
print("The test set contains {} images, in {} batches".format(len(test_dataloader.dataset), len(test_dataloader)))

The train set contains 60000 images, in 469 batches
The test set contains 10000 images, in 79 batches


In [44]:
from logging import log

class Net(nn.Module):
  def __init__(self):
    super().__init__()
    self.flatten = nn.Flatten()
    self.linear_relu = nn.Sequential(
        nn.Linear(28*28, 256),
        nn.ReLU(),       
        nn.Linear(256, 256),
        nn.ReLU(),
        nn.Linear(256, 128),
        nn.ReLU(),
        nn.Linear(128, 64),
        nn.ReLU(),
        nn.Linear(64,10),
    )

  def forward(self, x):
    x = self.flatten(x)
    logits = self.linear_relu(x)
    return logits


model = Net().to(device)
print(model)

loss_fn = nn.CrossEntropyLoss()
# loss_fn = nn.SoftMarginLoss()
# optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)
optimizer = torch.optim.SGD(model.parameters(), lr=0.01, momentum=0.85)

Net(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu): Sequential(
    (0): Linear(in_features=784, out_features=256, bias=True)
    (1): ReLU()
    (2): Linear(in_features=256, out_features=256, bias=True)
    (3): ReLU()
    (4): Linear(in_features=256, out_features=128, bias=True)
    (5): ReLU()
    (6): Linear(in_features=128, out_features=64, bias=True)
    (7): ReLU()
    (8): Linear(in_features=64, out_features=10, bias=True)
  )
)


In [45]:

def train(dataloader, model, loss_fn, optimizer):
  size = len(dataloader.dataset)
  model.train()

  for batch, (X,y) in enumerate(dataloader):
    X,y = X.to(device), y.to(device)

    pred = model(X)
    loss = loss_fn(pred,y)

    loss.backward()
    optimizer.step()
    optimizer.zero_grad()

    if batch % 100 == 0:
      loss, current = loss.item(), batch * len(X)
      print(f"loss: {loss:>7f} [{current:>5d}/{size:>5d}]")

def test(dataloader, model, loss_fn):
  size = len(dataloader.dataset)
  num_batches = len(dataloader)
  model.eval()
  test_loss, correct = 0,0

  with torch.inference_mode():
    for X,y in dataloader:
      X,y = X.to(device), y.to(device)
      pred = model(X)
      test_loss += loss_fn(pred, y).item()
      correct += (pred.argmax(1) == y).type(torch.float).sum().item()

  test_loss /= num_batches
  correct /= size
  print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")


In [46]:
epochs = 40

for t in range(epochs):
  print(f"Epoch {t+1}\n-------------------------------")
  train(train_dataloader, model, loss_fn, optimizer)
  test(test_dataloader, model, loss_fn)
print("Done!")

Epoch 1
-------------------------------
loss: 2.305113 [    0/60000]
loss: 2.255174 [12800/60000]
loss: 1.156485 [25600/60000]
loss: 0.810770 [38400/60000]
loss: 0.691693 [51200/60000]
Test Error: 
 Accuracy: 70.7%, Avg loss: 0.752694 

Epoch 2
-------------------------------
loss: 0.603482 [    0/60000]
loss: 0.553010 [12800/60000]
loss: 0.546334 [25600/60000]
loss: 0.551934 [38400/60000]
loss: 0.583819 [51200/60000]
Test Error: 
 Accuracy: 79.8%, Avg loss: 0.568236 

Epoch 3
-------------------------------
loss: 0.418007 [    0/60000]
loss: 0.422120 [12800/60000]
loss: 0.434360 [25600/60000]
loss: 0.458837 [38400/60000]
loss: 0.480729 [51200/60000]
Test Error: 
 Accuracy: 81.3%, Avg loss: 0.509917 

Epoch 4
-------------------------------
loss: 0.338948 [    0/60000]
loss: 0.381669 [12800/60000]
loss: 0.362062 [25600/60000]
loss: 0.413312 [38400/60000]
loss: 0.431055 [51200/60000]
Test Error: 
 Accuracy: 82.3%, Avg loss: 0.483687 

Epoch 5
-------------------------------
loss: 0.2979

In [47]:
# Save the model
torch.save(model.state_dict(), 'fashion_mnist_model.pth')

print('Model trained and saved.')

Model trained and saved.


In [40]:
# Load the saved model
model = Net().to(device)
model.load_state_dict(torch.load('fashion_mnist_model.pth', weights_only=True))
model.eval()

from PIL import Image, ImageOps

image_path = 'z.png'
image = Image.open(image_path).convert('L')
# image = ImageOps.invert(image_inv)
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Resize((28, 28))
])
image = transform(image).to(device)
print(f"Image shape: {image.shape}")


# Make a prediction 
with torch.no_grad():
    output = model(image)
    prediction = torch.argmax(output).item()

    print(f'Predicted digit: {prediction}')
    print(f"Fashion: {labels_dict[prediction]}")
    print("model:\n{}".format(model))
    print("dir(model):\n{}".format(dir(model)))


Image shape: torch.Size([1, 28, 28])
Predicted digit: 0
Fashion: T-shirt/top
model:
Net(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu): Sequential(
    (0): Linear(in_features=784, out_features=256, bias=True)
    (1): ReLU()
    (2): Linear(in_features=256, out_features=256, bias=True)
    (3): ReLU()
    (4): Linear(in_features=256, out_features=128, bias=True)
    (5): ReLU()
    (6): Linear(in_features=128, out_features=64, bias=True)
    (7): ReLU()
    (8): Linear(in_features=64, out_features=10, bias=True)
  )
)
dir(model):
['T_destination', '__annotations__', '__call__', '__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattr__', '__getattribute__', '__getstate__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__setstate__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', '_apply